In [ ]:
from google.cloud import storage
from google.cloud import bigquery
import os
import json


In [ ]:
# Configuration

GCP_PROJECT_ID = ! gcloud config get project
GCP_PROJECT_ID = GCP_PROJECT_ID[0]
LOCATION = "us-central1"
IMPORT_BUCKET_NAME = f"{GCP_PROJECT_ID}-lab-data-import"
ENTRY_GROUP_NAME = "databricks"
IMPORT_FILE = "databricks_import.json"

In [ ]:
import google.auth
from google.auth.transport.requests import AuthorizedSession
from requests import HTTPError
from typing import Any, Optional, Dict

def call_google_api(
    url: str,
    http_verb: str,
    request_body: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    creds, project = google.auth.default(
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    authed_session = AuthorizedSession(creds)
    try:
        response = authed_session.request(
            method=http_verb,
            url=url,
            json=request_body  # requests handles None for json param gracefully
        )

        response.raise_for_status()

        if response.status_code == 204:
            return {}

        return response.json()

    except HTTPError as e:
        # Provide more structured error information
        error_message = f"API call failed with status {e.response.status_code}: {e.response.text}"
        print(error_message) # Or use logging
        raise RuntimeError(error_message) from e

In [ ]:
def create_aspect_types():
  aspect_types = [
      "databricks-metastore",
      "databricks-catalog",
      "databricks-schema",
      "databricks-table",
      "databricks-view",
      "databricks-functions",
      "databricks-models",
      "databricks-volumes"
  ]


  for at in aspect_types:
    request_body = {
        "displayName": f"Databricks Metadata Marker for {at}",
        "description": "A simple marker for assets sourced from Databricks environments.",
        "metadataTemplate": {
        "name": f"{at}",
        "type": "record"
      }
    }
    print(request_body)

    url = f"https://dataplex.googleapis.com/v1/projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes?aspectTypeId={at}"
    response = call_google_api(url, "POST", request_body)
    pretty_json = json.dumps(response, indent=4, sort_keys=True)
    print(pretty_json)

create_aspect_types()


In [ ]:
def create_duc_entry():
  #create dataplex universal catalog entry group
  ! gcloud dataplex entry-groups create {ENTRY_GROUP_NAME} --location={LOCATION}

create_duc_entry()

In [ ]:
def create_duc_entry_types():
  #create dataplex universal catalog entry type

  entry_types = [
      "databricks-metastore",
      "databricks-catalog",
      "databricks-schema",
      "databricks-table",
      "databricks-view",
      "databricks-functions",
      "databricks-models",
      "databricks-volumes"
  ]

  for et in entry_types:
    !gcloud dataplex entry-types create {et} --location={LOCATION}

create_duc_entry_types()

In [ ]:
#create bucket if it does not exist
def create_storage_bucket():
  storage_client = storage.Client(project=GCP_PROJECT_ID)
  buckets = storage_client.list_buckets()
  bucket_names = [bucket.name for bucket in buckets]

  bucket = storage_client.bucket(IMPORT_BUCKET_NAME)

  if not bucket.exists():
      try:
          bucket = storage_client.create_bucket(IMPORT_BUCKET_NAME)
          print(f"Bucket {bucket.name} created.")
      except Exception as e:
          print(f"Error creating bucket: {e}")
  else:
      print(f"Bucket {IMPORT_BUCKET_NAME} already exists.")

create_storage_bucket()

In [ ]:
# define import payload
dataplex_import_payload_json = f"""
{{"entry": {{"name": "projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2", "entryType": "projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-metastore", "aspects": {{"{GCP_PROJECT_ID}.{LOCATION}.databricks-metastore": {{"aspectType": "{GCP_PROJECT_ID}.{LOCATION}.databricks-metastore", "data": {{}}, "path": ""}}}}, "fullyQualifiedName": "custom:`metastore_aws_us_east_2`", "parentEntry": ""}}, "aspectKeys": ["{GCP_PROJECT_ID}.{LOCATION}.databricks-metastore"], "updateMask": "aspects"}}
{{"entry": {{"name": "projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples", "entryType": "projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-catalog", "aspects": {{"{GCP_PROJECT_ID}.{LOCATION}.databricks-catalog": {{"aspectType": "{GCP_PROJECT_ID}.{LOCATION}.databricks-catalog", "data": {{}}, "path": ""}}}}, "parentEntry": "projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2", "fullyQualifiedName": "custom:`metastore_aws_us_east_2`.samples"}}, "aspectKeys": ["{GCP_PROJECT_ID}.{LOCATION}.databricks-catalog"], "updateMask": "aspects"}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/tpch","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.tpch","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples","entrySource":{{"displayName":"tpch","system":"custom"}},"aspects":{{"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-schema"}},"aspectKeys":["{GCP_PROJECT_ID}.{LOCATION}.databricks-schema"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.bakehouse","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples","entrySource":{{"displayName":"bakehouse","system":"custom"}},"aspects":{{"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-schema"}},"aspectKeys":["{GCP_PROJECT_ID}.{LOCATION}.databricks-schema"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/healthverity","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.healthverity","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples","entrySource":{{"displayName":"healthverity","system":"custom"}},"aspects":{{"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-schema"}},"aspectKeys":["{GCP_PROJECT_ID}.{LOCATION}.databricks-schema"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/accuweather","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.accuweather","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples","entrySource":{{"displayName":"accuweather","system":"custom"}},"aspects":{{"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-schema"}},"aspectKeys":["{GCP_PROJECT_ID}.{LOCATION}.databricks-schema"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/nyctaxi","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.nyctaxi","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples","entrySource":{{"displayName":"nyctaxi","system":"custom"}},"aspects":{{"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-schema","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-schema"}},"aspectKeys":["{GCP_PROJECT_ID}.{LOCATION}.databricks-schema"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/tpch/tables/customer","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.tpch.customer","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/tpch","entrySource":{{"displayName":"customer","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"c_custkey","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"c_name","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"c_address","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"c_nationkey","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"c_phone","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"c_acctbal","mode":"REQUIRED","dataType":"DECIMAL","metadataType":"NUMBER"}},{{"name":"c_mktsegment","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"c_comment","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse/tables/media_customer_reviews","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.bakehouse.media_customer_reviews","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse","entrySource":{{"displayName":"media_customer_reviews","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"review","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"franchiseID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"review_date","mode":"REQUIRED","dataType":"TIMESTAMP","metadataType":"TIMESTAMP"}},{{"name":"new_id","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse/tables/media_gold_reviews_chunked","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.bakehouse.media_gold_reviews_chunked","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse","entrySource":{{"displayName":"media_gold_reviews_chunked","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"franchiseID","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"review_date","mode":"REQUIRED","dataType":"TIMESTAMP","metadataType":"TIMESTAMP"}},{{"name":"chunked_text","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"chunk_id","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"review_uri","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse/tables/sales_customers","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.bakehouse.sales_customers","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse","entrySource":{{"displayName":"sales_customers","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"customerID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"first_name","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"last_name","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"email_address","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"phone_number","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"address","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"city","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"state","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"country","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"continent","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"postal_zip_code","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"gender","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse/tables/sales_franchises","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.bakehouse.sales_franchises","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse","entrySource":{{"displayName":"sales_franchises","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"franchiseID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"name","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"city","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"district","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"zipcode","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"country","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"size","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"longitude","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"latitude","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"supplierID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse/tables/sales_suppliers","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.bakehouse.sales_suppliers","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse","entrySource":{{"displayName":"sales_suppliers","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"supplierID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"name","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"ingredient","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"continent","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"city","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"district","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"size","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"longitude","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"latitude","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"approved","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse/tables/sales_transactions","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.bakehouse.sales_transactions","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/bakehouse","entrySource":{{"displayName":"sales_transactions","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"transactionID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"customerID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"franchiseID","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"dateTime","mode":"REQUIRED","dataType":"TIMESTAMP","metadataType":"TIMESTAMP"}},{{"name":"product","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"quantity","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"unitPrice","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"totalPrice","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}},{{"name":"paymentMethod","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"cardNumber","mode":"REQUIRED","dataType":"LONG","metadataType":"NUMBER"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/accuweather/tables/forecast_daily_calendar_imperial","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.accuweather.forecast_daily_calendar_imperial","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/accuweather","entrySource":{{"displayName":"forecast_daily_calendar_imperial","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"city_name","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"country_code","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"latitude","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"longitude","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"date","mode":"REQUIRED","dataType":"DATE","metadataType":"DATETIME"}},{{"name":"cloud_cover_perc_avg","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"cloud_cover_perc_max","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"cloud_cover_perc_min","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"degree_days_cooling","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"degree_days_freezing","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"degree_days_growing","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"degree_days_heating","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"humidity_relative_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"humidity_relative_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"humidity_relative_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"index_air_quality_24hr_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"index_uv_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"index_uv_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"index_uv_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"minutes_of_ice_total","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"minutes_of_precipitation_total","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"minutes_of_sun_total","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"minutes_of_rain_total","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"minutes_of_snow_total","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"has_ice","mode":"REQUIRED","dataType":"BOOLEAN","metadataType":"BOOLEAN"}},{{"name":"ice_lwe_rate_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"ice_lwe_rate_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"ice_lwe_rate_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"ice_lwe_total","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"ice_probability","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"has_precipitation","mode":"REQUIRED","dataType":"BOOLEAN","metadataType":"BOOLEAN"}},{{"name":"precipitation_lwe_rate_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"precipitation_lwe_rate_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"precipitation_lwe_rate_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"precipitation_lwe_total","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"precipitation_probability","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"precipitation_type_desc_predominant","mode":"REQUIRED","dataType":"STRING","metadataType":"STRING"}},{{"name":"has_rain","mode":"REQUIRED","dataType":"BOOLEAN","metadataType":"BOOLEAN"}},{{"name":"rain_lwe_rate_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"rain_lwe_rate_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"rain_lwe_rate_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"rain_lwe_total","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"rain_probability","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"snow_liquid_ratio_accuweather_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_liquid_ratio_accuweather_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_liquid_ratio_accuweather_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"has_snow","mode":"REQUIRED","dataType":"BOOLEAN","metadataType":"BOOLEAN"}},{{"name":"snow_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_total","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_lwe_rate_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_lwe_rate_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_lwe_rate_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_lwe_total","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"snow_probability","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"solar_irradiance_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"solar_irradiance_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"solar_irradiance_total","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_dew_point_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_dew_point_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_dew_point_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_heat_index_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_heat_index_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_heat_index_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_realfeel_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_realfeel_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_realfeel_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_realfeel_shade_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_realfeel_shade_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_realfeel_shade_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_wind_chill_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_wind_chill_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"temperature_wind_chill_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"visibility_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"visibility_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"visibility_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_direction_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_gust_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_gust_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_gust_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_gust_direction_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_speed_avg","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_speed_max","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"wind_speed_min","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
{{"entry":{{"name":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/nyctaxi/tables/trips","fullyQualifiedName":"custom:`metastore_aws_us_east_2`.samples.nyctaxi.trips","parentEntry":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks/entries/metastore_aws_us_east_2/catalog/samples/schemas/nyctaxi","entrySource":{{"displayName":"trips","system":"custom"}},"aspects":{{"dataplex-types.global.schema":{{"aspectType":"dataplex-types.global.schema","data":{{"fields":[{{"name":"tpep_pickup_datetime","mode":"REQUIRED","dataType":"TIMESTAMP","metadataType":"TIMESTAMP"}},{{"name":"tpep_dropoff_datetime","mode":"REQUIRED","dataType":"TIMESTAMP","metadataType":"TIMESTAMP"}},{{"name":"trip_distance","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"fare_amount","mode":"REQUIRED","dataType":"DOUBLE","metadataType":"NUMBER"}},{{"name":"pickup_zip","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}},{{"name":"dropoff_zip","mode":"REQUIRED","dataType":"INT","metadataType":"NUMBER"}}]}}}},"{GCP_PROJECT_ID}.{LOCATION}.databricks-table":{{"aspectType":"{GCP_PROJECT_ID}.{LOCATION}.databricks-table","data":{{}}}}}},"entryType":"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table"}},"aspectKeys":["dataplex-types.global.schema","{GCP_PROJECT_ID}.{LOCATION}.databricks-table"],"updateMask":["aspects"]}}
"""

#load csv file to gcs storage
print(dataplex_import_payload_json)
storage_client = storage.Client(project=GCP_PROJECT_ID)
storage_client.bucket(IMPORT_BUCKET_NAME).blob(IMPORT_FILE).upload_from_string(dataplex_import_payload_json)
print(f"CSV file '{IMPORT_FILE}' loaded to GCS Bucket '{IMPORT_BUCKET_NAME}'.")


In [ ]:


request_body = {
    "type": "IMPORT",
    "import_spec": {
        "source_storage_uri": f"gs://{IMPORT_BUCKET_NAME}/",
        "entry_sync_mode": "FULL",
        "aspect_sync_mode": "INCREMENTAL",
        "log_level": "DEBUG",
        "scope": {
            "entry_groups": [
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryGroups/databricks"
            ],
            "entry_types": [
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-metastore",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-catalog",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-schema",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-table",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-view",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-functions",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-models",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/entryTypes/databricks-volumes"
            ],
            "aspect_types": [
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-metastore",
                "projects/dataplex-types/locations/global/aspectTypes/schema",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-catalog",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-schema",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-table",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-view",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-functions",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-models",
                f"projects/{GCP_PROJECT_ID}/locations/{LOCATION}/aspectTypes/databricks-volumes"
            ]
        }
    }
}


In [ ]:

url = f"https://dataplex.googleapis.com/v1/projects/{GCP_PROJECT_ID}/locations/{LOCATION}/metadataJobs"
response = call_google_api(url, "POST", request_body)
metadata_job_target = response['metadata']['target']
pretty_json = json.dumps(response, indent=4, sort_keys=True)
print(pretty_json)

In [ ]:
status_url = f"https://dataplex.googleapis.com/v1/{metadata_job_target}"
response = call_google_api(status_url, "GET")
pretty_json = json.dumps(response, indent=4, sort_keys=True)
print(pretty_json)